In [ ]:
import pandas as pd
import numpy as np
import json
import os

print("🚀 Starting Export from 'train_pos_weight.txt'...")

# Load user embeddings - format: {"0": [0.1, 0.2, ...], "1": [...]}
with open('user_vectors.json', 'r') as f:
    user_vectors_data = json.load(f)
    # Sort by numeric key to ensure correct order
    sorted_indices = sorted([int(k) for k in user_vectors_data.keys()])
    user_final_embs = np.array([user_vectors_data[str(i)] for i in sorted_indices])
print(f"✅ Loaded {len(user_final_embs)} user embeddings from user_vectors.json")

# Load item embeddings - same format
with open('item_vectors.json', 'r') as f:
    item_vectors_data = json.load(f)
    sorted_indices = sorted([int(k) for k in item_vectors_data.keys()])
    item_final_embs = np.array([item_vectors_data[str(i)] for i in sorted_indices])
print(f"✅ Loaded {len(item_final_embs)} item embeddings from item_vectors.json")

# 1. ĐỌC FILE INTERACTION CỦA MODEL
# Giả sử file nằm ở thư mục hiện tại hoặc đường dẫn cụ thể
file_path = 'train_pos_weight.txt' # Sửa đường dẫn nếu cần

# Đọc file không header, phân tách bằng khoảng trắng hoặc tab
df_interactions = pd.read_csv(file_path, header=None, sep='\s+', names=['user_idx', 'item_idx', 'weight'])

# Map trọng số sang loại tương tác (theo yêu cầu của bạn)
def map_weight_to_type(w):
    if w >= 3.0: return 'like' # Hoặc share
    return 'click'

df_interactions['type'] = df_interactions['weight'].apply(map_weight_to_type)

print(f"📊 Loaded {len(df_interactions)} interactions from text file.")

# 2. LẤY MẪU 300 DÒNG (SMART SUBSET)
# Lấy 300 dòng ngẫu nhiên để làm Demo
small_df = df_interactions.sample(n=300, random_state=42).copy()

# Lấy danh sách các index cần thiết
needed_user_indices = set(small_df['user_idx'].unique())
needed_item_indices = set(small_df['item_idx'].unique())

print(f"⚡ Subset contains {len(needed_user_indices)} Users and {len(needed_item_indices)} Items.")

# 3. EXPORT USER DATA (Chỉ những user có trong 300 dòng này)
user_export_list = []

# raw_user_ids: List ID gốc bạn đã load từ user_list.json (nếu có)
# Nếu không có file map gốc, ta dùng luôn index làm ID giả
for idx in needed_user_indices:
    # Lấy vector từ model (user_final_embs đã tính ở các bước trước)
    if idx < len(user_final_embs):
        vector = user_final_embs[idx]
        
        # Cố gắng lấy ID gốc, nếu không có thì dùng user_IDModel
        real_uid_str = str(raw_user_ids[idx]) if 'raw_user_ids' in locals() and idx < len(raw_user_ids) else f"u_{idx}"
        
        user_export_list.append({
            "tenrec_uid": real_uid_str,
            "model_index": int(idx), # Quan trọng: Phải là int
            "embedding": vector.tolist(),
            "preferences": {"tech": 0.1, "sports": 0.1}
        })

with open('user_data_full.json', 'w') as f:
    json.dump(user_export_list, f)
print("✅ Exported user_data_full.json")

# 4. EXPORT ITEM DATA (Chỉ những item có trong 300 dòng này)
item_export_list = []

for idx in needed_item_indices:
    if idx < len(item_final_embs):
        vector = item_final_embs[idx]
        
        real_iid_str = str(raw_item_ids[idx]) if 'raw_item_ids' in locals() and idx < len(raw_item_ids) else f"i_{idx}"
        
        item_export_list.append({
            "tenrec_id": real_iid_str,
            "model_index": int(idx),
            "embedding": vector.tolist(),
            "category": 0, # Mặc định, sẽ được map sang tên ở Node.js
            "click_count": int(np.random.randint(100, 5000)),
            "like_count": int(np.random.randint(10, 500))
        })

with open('item_data_full.json', 'w') as f:
    json.dump(item_export_list, f)
print("✅ Exported item_data_full.json")

# 5. EXPORT INTERACTION DATA (Chính là 300 dòng từ file txt)
interaction_export_list = []

for _, row in small_df.iterrows():
    interaction_export_list.append({
        "user_idx": int(row['user_idx']), # ID của Model
        "item_idx": int(row['item_idx']), # ID của Model
        "type": row['type']               # click hoặc like
    })

with open('interaction_data.json', 'w') as f:
    json.dump(interaction_export_list, f)
print("✅ Exported interaction_data.json")

print("\n🎉 DONE! Copy 3 file JSON này vào thư mục 'framework/data/'")